In [4]:
from azureml.core import Dataset
from azureml.core import Workspace
import datetime

ws = Workspace.from_config()
dataset = Dataset.get_by_name(ws, name='BTC-Bitstamp-raw')

df = dataset.to_pandas_dataframe()
df = df.rename(columns={"Timestamp": "dateid", "Close": "price","Volume_(BTC)":"volume"})


df

,dateid,Open,High,Low,price,volume,Volume_(Currency),Weighted_Price
0,1325317920,4.39,4.39,4.39,4.39,0.455581,2.000000,4.390000
1,1325317980,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1325318040,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1325318100,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1325318160,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
4857372,1617148560,58714.31,58714.31,58686.00,58686.00,1.384487,81259.372187,58692.753339
4857373,1617148620,58683.97,58693.43,58683.97,58685.81,7.294848,428158.146640,58693.226508
4857374,1617148680,58693.43,58723.84,58693.43,58723.84,1.705682,100117.070370,58696.198496
4857375,1617148740,58742.18,58770.38,58742.18,58760.59,0.720415,42332.958633,58761.866202


In [5]:
dateparse = lambda x: datetime.datetime.utcfromtimestamp(float(x))
df['date'] = [dateparse(x) for x in df['dateid']]
df = df.set_index('date')
del df['dateid']
df['ask'] = df['price']*1.01
df['bid'] = df['price']*0.99
df = df.reindex(columns=['price','ask','bid','volume','High','Low'])
df

,price,ask,bid,volume,High,Low
date,,,,,,
2011-12-31 07:52:00,4.39,4.4339,4.3461,0.455581,4.39,4.39
2011-12-31 07:53:00,NaN,NaN,NaN,NaN,NaN,NaN
2011-12-31 07:54:00,NaN,NaN,NaN,NaN,NaN,NaN
2011-12-31 07:55:00,NaN,NaN,NaN,NaN,NaN,NaN
2011-12-31 07:56:00,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2021-03-30 23:56:00,58686.00,59272.8600,58099.1400,1.384487,58714.31,58686.00
2021-03-30 23:57:00,58685.81,59272.6681,58098.9519,7.294848,58693.43,58683.97
2021-03-30 23:58:00,58723.84,59311.0784,58136.6016,1.705682,58723.84,58693.43


In [6]:
#df = df.dropna()

def addVals(timeframe):
    p=df['price']
    df['ma'+timeframe]=(p/p.rolling(timeframe).mean())-1
    mins = df['Low'].rolling(timeframe).min()
    df['maxmin'+timeframe]=(p - mins)/(df['High'].rolling(timeframe).max() - mins) 
    df['std'+timeframe] =  p.rolling(timeframe).std()/p

addVals('15min')
#addVals('1h')
#addVals('10h')
#addVals('1D')
#addVals('10D')
#addVals('20D')

df = df.dropna()

    
del df['High']
del df['Low']

df

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/anaconda/envs/azureml_py36/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats

,price,ask,bid,volume,ma15min,maxmin15min,std15min
date,,,,,,,
2011-12-31 17:00:00,4.58,4.6258,4.5342,9.000000,0.001093,1.000000,0.001544
2012-01-03 17:14:00,5.14,5.1914,5.0886,0.020000,-0.014382,0.000000,0.020635
2012-01-03 17:26:00,5.29,5.3429,5.2371,11.000000,0.014382,1.000000,0.020050
2012-01-03 17:27:00,5.29,5.3429,5.2371,4.010815,0.009542,1.000000,0.016371
2012-01-04 15:53:00,5.32,5.3732,5.2668,0.186090,0.012369,1.000000,0.017279
...,...,...,...,...,...,...,...
2021-03-30 23:56:00,58686.00,59272.8600,58099.1400,1.384487,0.000311,0.530095,0.000776
2021-03-30 23:57:00,58685.81,59272.6681,58098.9519,7.294848,0.000229,0.528927,0.000744
2021-03-30 23:58:00,58723.84,59311.0784,58136.6016,1.705682,0.000736,0.762742,0.000707
